In [1]:
f = open("to_download.txt", "r")
s = f.read()

In [6]:
s.split("\n")

In [16]:
ids = [l.split(" ")[11] for l in s.split("\n") if "Failed to download id" in l]

In [ ]:
"""
Downloading all time series in the provided JSON file.
""" 
fred.key("01571df9f62cb8c1ff7dcb8471b164c8")
#num_requests = 0
tot_downloaded = 0
num_files_written = 0
list_json = []
request_time_stamps = []
#curr_dir = f"dir{tot_downloaded // cfg.source.files_per_folder :04d}/"

dirs = glob.glob(cfg.source.path.FRED.meta + "/*")
for d in dirs:
    file_names = glob.glob(d + "/*")
    for fname in file_names:
        with open(fname, "r") as fp:
            ids_meta = json.load(fp)
            fp.close()

        for id_meta in ids_meta:
            try:
                observations = fred.observations(id_meta["id"])
                request_time_stamps.append(time.time())
                #num_requests += 1
                ts = {
                    "id" : id_meta["id"],
                    "source" : id_meta["source"],
                    "node_id" : id_meta["node_id"],
                    "category_name" : id_meta["category_name"],
                    "parent_id" : id_meta["parent_id"],
                    "frequency" : id_meta["frequency"],
                    "observations" : [{"date" : obs["date"], "value" : obs["value"]} for obs in observations["observations"]]
                }

                tot_downloaded += 1
                list_json.append(ts)
                if len(list_json) > cfg.source.samples_per_json:
                    filename = f"raw_{num_files_written:>06}.json"
                    if num_files_written % cfg.source.files_per_folder == 0:
                        curr_dir = f"dir{num_files_written // cfg.source.files_per_folder :04d}/"
                        os.makedirs(os.path.join(cfg.source.path.FRED.raw, curr_dir), exist_ok=True)
                    with open(os.path.join(*[cfg.source.path.FRED.raw, curr_dir, filename]), "w") as fp:
                        json.dump(list_json, fp, sort_keys=True, indent=4, separators=(",", ": "))
                        fp.close()
                    
                    with open(os.path.join(cfg.source.path.FRED.meta, "ids_downloaded.txt"), "a") as fp:
                        for j in list_json:
                            fp.write(j["id"])
                            fp.write("\n")
                        fp.close()

                    num_files_written += 1
                    list_json = []

                if tot_downloaded % 10000 == 0:
                    logger.info(f"Downloaded {tot_downloaded} time series.")

            except Exception as e:
                logger.info(f"Failed to download id {id_meta['id']} from fname {fname}.")
                logger.warning(e)

            if len(request_time_stamps) > rate_limit:
                first = request_time_stamps.pop(0)
                if time.time() - first < sleep_time:
                    #logger.info(f"Sleeping for {request_time_stamps[0]-first}.")
                    time.sleep(request_time_stamps[0]-first)
    
    logger.info(f"Written files in directory {d} and currently have {tot_downloaded:>6} time series saved")

In [31]:
import fred
import time
fred.key("01571df9f62cb8c1ff7dcb8471b164c8")

In [32]:
cnt = 0
list_json = []
for id in ids:
    d = fred.series(id)
    o = fred.observations(id)
    cnt += 2
    ts = {
        "id" : id,
        "source" : "FRED",
        "node_id" : "NA",
        "category_name" : "NA",
        "parent_id" : "NA",
        "frequency" : d["seriess"][0]["frequency"],
        "observations" : [{"date" : obs["date"], "value" : obs["value"]} for obs in o["observations"]]
    }
    list_json.append(ts)
    if cnt > 55:
        print("Sleeping.")
        time.sleep(60)
        cnt = 0

Sleeping.
Sleeping.
Sleeping.


In [33]:
print(len(list_json))

88


In [41]:
import json
import sys
sys.path.append("")
sys.path.append("../..")
with open("../GPTime/data/raw/FRED/dir0003/raw_000343.json", "a") as fp:
    json.dump(list_json, fp, sort_keys=True, indent=4, separators=(",", ": "))
    fp.close()

In [34]:
import os
os.getcwd()

'/home/shomea/e/erikde/Documents/GPTime-Series/notebooks'